In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics

In [2]:
def normal_eqn(df,deg):
    df2=df.copy()
    for i in range(len(deg)):
        dpow=np.power(df2.iloc[:,i],deg[i])
        df2.iloc[:,i]=dpow
    n=len(df2.columns)-1
    D=df2.iloc[:,0:n].to_numpy()
    D=np.insert(D,0,1,axis=1)
    R=df2.iloc[:,n].to_numpy()
    DT=np.transpose(D)
    DTD=np.matmul(DT,D)
    if np.linalg.det(DTD)==0:
        return 0,0
    DTDinv=np.linalg.inv(DTD)
    DTR=np.matmul(DT,R)
    W=np.matmul(DTDinv,DTR)
    gx=W[0]
    for i in range(n):
        gx+=W[i+1]*df2.iloc[:,i]
        i+=1
    gx=gx.tolist()
    return W,gx

In [3]:
def best_hypothesis(df):
    df2=df.copy()
    n=len(df2.columns)-1
    deg=[]
    for i in range(n):
        deg.append(1)
    final_deg=[]
    for i in range(n):
        deg2=deg.copy()
        W,gx=normal_eqn(df2,deg2)
        if gx==0:
            continue
        rmse=metrics.mean_squared_error(df2.iloc[:,n],gx)
        d=1
        deg2[i]=0.9
        W,gx=normal_eqn(df2,deg2)
        if gx==0:
            continue
        rtemp=metrics.mean_squared_error(df2.iloc[:,n],gx)
        if rtemp<rmse:
            rmse=rtemp
            d=deg2[i]
            while(deg2[i]>0):
                deg2[i]=round((deg2[i]-0.1),2)
                W,gx=normal_eqn(df2,deg2)
                if gx==0:
                    continue
                rtemp=metrics.mean_squared_error(df2.iloc[:,n],gx)
                if rtemp<rmse:
                    rmse=rtemp
                    d=deg2[i]
                else:
                    break
        else:
            deg2[i]=1
            W,gx=normal_eqn(df2,deg2)
            rmse=metrics.mean_squared_error(df2.iloc[:,n],gx)
            while(deg2[i]<20):
                deg2[i]+=1
                W,gx=normal_eqn(df2,deg2)
                if gx==0:
                    continue
                rtemp=metrics.mean_squared_error(df2.iloc[:,n],gx)
                if rtemp<rmse:
                    rmse=rtemp
                    d=deg2[i]
                else:
                    dtemp=deg2[i]+1
                    deg2[i]=d
                    while(deg2[i]<dtemp):
                        deg2[i]=round(deg2[i]+0.1,2)
                        W,gx=normal_eqn(df2,deg2)
                        if gx==0:
                            continue
                        rtemp=metrics.mean_squared_error(df2.iloc[:,n],gx)
                        if rtemp<rmse:
                            rmse=rtemp
                            d=deg2[i]
                            continue
                        else:
                            break
                    break
        final_deg.append(d)
    return final_deg
                    

In [4]:
df=pd.read_csv("Real estate.csv")
df.head()

,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


In [5]:
deg=best_hypothesis(df)
deg

[20, 0.3, 0.1, 2, 20, 0.1]

In [6]:
W,gx=normal_eqn(df,deg)
gx

[52.1675612045874,
 45.31574633513446,
 47.002696667743294,
 46.42833517297004,
 44.517843523926786,
 25.660211142352637,
 35.2816608553494,
 47.03884294313502,
 15.527417627425166,
 30.63282803711445,
 37.028238726567,
 56.79195369695117,
 37.09559693360643,
 21.229532120154545,
 42.345655117431306,
 40.84016533853537,
 56.97301027592039,
 38.72235691725109,
 43.90182717784592,
 58.5653333594837,
 31.63426613916272,
 48.54804923401571,
 25.468023920842825,
 46.35723377208524,
 35.56156409805408,
 29.787070794016472,
 44.95770996214196,
 41.06099575116059,
 41.12277014628489,
 43.47581761627225,
 17.205641953903978,
 36.07474117197489,
 33.22240766577306,
 45.35417789448002,
 46.98159356830365,
 38.7715569264783,
 26.095944431739554,
 27.83358026082533,
 40.48713733424847,
 46.0293543441785,
 15.061521662864834,
 18.224011723119474,
 35.86744128963255,
 38.834783830991,
 47.369922552959906,
 37.621623007404196,
 43.12569065602656,
 38.45362123398809,
 16.360506292072387,
 11.2907344788

In [7]:
metrics.mean_squared_error(df.iloc[:,6],gx)

61.58870411246215

In [6]:
df=pd.read_csv("boston.csv")
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [7]:
deg=best_hypothesis(df)
deg

[2, 6, 4, 1, 2.2, 6.5, 20, 0.1, 0.6, 0.1, 0.1, 0.3, 0.1]

In [8]:
W,gx=normal_eqn(df,deg)
gx

[29.710442536783987,
 23.755224195250975,
 32.98714347507105,
 33.72987971781292,
 29.853097291715514,
 27.489254471973368,
 20.974612951893704,
 17.40280807546975,
 12.593313089108818,
 17.269529626674057,
 16.836727066224213,
 19.822237869461887,
 18.825900312314232,
 20.186765599078626,
 18.884751856131743,
 20.027662433337113,
 22.204200054076125,
 15.895775752232467,
 17.262854243489073,
 18.33800896364974,
 13.020151528350937,
 16.739417130435612,
 14.515573627266448,
 13.707916722631296,
 14.879256182786136,
 13.710368248186015,
 15.057367097734769,
 14.026567342386628,
 18.129764267730295,
 19.31016707143192,
 11.70549061242221,
 17.672077578299096,
 9.727119773278645,
 14.16123442713662,
 13.489545845294842,
 23.198890485264826,
 21.5662425909033,
 23.08271454626899,
 22.173159841551225,
 30.918903713424115,
 38.22168462743585,
 29.88276507306503,
 26.8159676816921,
 24.96413005792739,
 22.605210794412713,
 22.07086302388967,
 19.43323390415948,
 16.763922307270747,
 11.425352

In [9]:
metrics.mean_squared_error(df.iloc[:,13],gx)

14.777593903706233

In [10]:
df=pd.read_csv("CyclePowerPlant.csv")
df.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [11]:
deg=best_hypothesis(df)
deg

[0.6, 0.1, 0.1, 0.6]

In [12]:
W,gx=normal_eqn(df,deg)
gx

[466.74786396964834,
 443.69672516269105,
 487.3436156838961,
 449.0971936894067,
 472.7758207202567,
 442.1145690124938,
 462.8301118699384,
 477.6338743524019,
 470.2061559817246,
 472.01897089161304,
 458.4555333600127,
 455.5016445491317,
 438.5394717927117,
 442.8863210013631,
 435.98262972802996,
 457.3859014016227,
 462.4955907546702,
 474.2006221450196,
 465.2395371061242,
 471.2813982953192,
 460.4583717530511,
 490.135665289318,
 479.55590441595905,
 441.2580137550952,
 443.1704274425289,
 435.2225369015221,
 467.07994092753063,
 481.0943667415099,
 486.7589539733336,
 436.58619241976675,
 447.4735728532926,
 442.08510405695756,
 434.937287974067,
 434.1232044492653,
 445.9011347011954,
 466.70368045626816,
 441.4826707015582,
 458.72138856960436,
 468.4678279112284,
 446.947364212945,
 449.1075435967122,
 467.18381724910034,
 477.55738874374975,
 469.9573108024199,
 475.3138738615229,
 449.2869519175176,
 465.23127476778126,
 454.86615750238457,
 470.62684242332534,
 479.297

In [13]:
metrics.mean_squared_error(df.iloc[:,4],gx)

19.168016110092186